In [1]:
import numpy as np
import plotly as pl
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv('../data/rovbase/rovviltskader.csv')
df['year'] = pd.to_datetime(df['Skadedato_fra']).dt.year

df = df.astype({"year": str}) # må konvertere til string for at de skal vises seperatat på kartet

fig = px.scatter_mapbox(df, lat='latitude', lon='longitude', color='year', opacity=0.8, width=800, height=500,
                        zoom=6, title='Rovviltskader Meråker 2015-2022',
                        mapbox_style="stamen-terrain")
fig.show()

In [3]:
df = df.astype({"Skadearsak": str}) # må konvertere til string for at de skal vises seperatat på kartet
#df2 = df.groupby(['year', 'Skadearsak'])['Skadearsak'].count()

fig = px.bar(df, x='year', color="Skadearsak", barmode = 'stack')
fig.show()